In [1]:
from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession
from delta.tables import DeltaTable

In [2]:
builder = SparkSession.builder \
    .appName("DeltaTableExample") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

24/06/23 20:29:37 WARN Utils: Your hostname, paulo resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/06/23 20:29:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/mnt/c/Users/paulo_ufva11y/OneDrive/Documentos/Estudos/data-engineering-articles/.venv/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/paulooctavio/.ivy2/cache
The jars for the packages stored in: /home/paulooctavio/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4a3e233a-ac72-43b8-ac13-59ed9be96684;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 535ms :: artifacts dl 12ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.2.0 from central in [default]
	io.delta#delta-storage;3.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |

In [3]:
# Create a sample DataFrame
data = [
    ("product_001", "electronics", 150.0),
    ("product_002", "books", 20.0),
    ("product_003", "clothing", 75.0),
    ("product_004", "electronics", 200.0),
    ("product_005", "books", 15.0),
]
columns = ["product_id", "category", "price"]
df = spark.createDataFrame(data, columns)
df.show()

+-----------+-----------+-----+
| product_id|   category|price|
+-----------+-----------+-----+
|product_001|electronics|150.0|
|product_002|      books| 20.0|
|product_003|   clothing| 75.0|
|product_004|electronics|200.0|
|product_005|      books| 15.0|
+-----------+-----------+-----+



In [5]:
df.write.format("delta").save("./tmp/delta-table")

In [6]:
delta_df = spark.read.format("delta").load("./tmp/delta-table")

In [7]:
delta_table = DeltaTable.forPath(spark, "./tmp/delta-table")

delta_table.update(
    condition = "product_id == 'product_001'",
    set = { "category": "'books'" }
)

24/06/23 20:35:11 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [8]:
updated_df = spark.read.format("delta").load("./tmp/delta-table")
updated_df.show()

+-----------+-----------+-----+
| product_id|   category|price|
+-----------+-----------+-----+
|product_004|electronics|200.0|
|product_003|   clothing| 75.0|
|product_001|      books|150.0|
|product_002|      books| 20.0|
|product_005|      books| 15.0|
+-----------+-----------+-----+



In [10]:
version_df = (
    spark.read.format("delta")
    .option("versionAsOf", 0)
    .load("./tmp/delta-table")
)

In [ ]:
version_df.show()

+-----------+-----------+-----+
| product_id|   category|price|
+-----------+-----------+-----+
|product_004|electronics|200.0|
|product_001|electronics|150.0|
|product_003|   clothing| 75.0|
|product_005|      books| 15.0|
|product_002|      books| 20.0|
+-----------+-----------+-----+

